In [1]:
from dataset import Dataset
from feature_extractor import FeatureExtractor
from monitors import GaussianMixtureMonitor
from evaluation import Evaluator

import torch

/home/jorisguerin/Projects/neural-network-monitoring-benchmark/monitors.py:164: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if self.n_clusters_type is not "auto":


In [13]:
batch_size = 100
device_name = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [14]:
id_dataset = "cifar10"
ood_dataset = "cifar10"

model = "densenet"
layer = 98

atk = "fgsm"

In [15]:
dataset_train = Dataset(id_dataset, "train", model, batch_size=batch_size)
dataset_test = Dataset(id_dataset, "test", model, batch_size=batch_size)
dataset_ood = Dataset(ood_dataset, "test", model, None, atk, batch_size=batch_size)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [16]:
feature_extractor = FeatureExtractor(model, id_dataset, [layer], device_name)

In [17]:
features_train, logits_train, softmax_train, \
    pred_train, lab_train = feature_extractor.get_features(dataset_train)
features_test, logits_test, softmax_test, \
    pred_test, lab_test = feature_extractor.get_features(dataset_test)
features_ood, logits_ood, softmax_ood, \
    pred_ood, lab_ood = feature_extractor.get_features(dataset_ood)

Extracting layers: 'relu'


100%|█████████████████████████████████████████| 500/500 [15:33<00:00,  1.87s/it]


Extracting layers: 'relu'


100%|█████████████████████████████████████████| 100/100 [03:11<00:00,  1.91s/it]


Extracting layers: 'relu'


100%|█████████████████████████████████████████| 100/100 [06:52<00:00,  4.13s/it]


In [18]:
eval_oms = Evaluator("oms", is_novelty=(id_dataset != ood_dataset))
eval_oms.fit_ground_truth(lab_test, lab_ood, pred_test, pred_ood)

In [19]:
monitor = GaussianMixtureMonitor(id_dataset, model, layer, n_components=5,
                                                         constraint="diag", is_cv=True)
monitor.fit(features_train[0], pred_train, lab_train, save=True)

scores_test = monitor.predict(features_test[0], pred_test)
scores_ood = monitor.predict(features_ood[0], pred_ood)

aupr = eval_oms.get_average_precision(-scores_test, -scores_ood)
auroc = eval_oms.get_auroc(-scores_test, -scores_ood)
tnr95tpr = eval_oms.get_tnr_frac_tpr_oms(-scores_test, -scores_ood, frac=0.95)

In [20]:
print(aupr, auroc, tnr95tpr)

0.6034687443705132 0.8397250602508426 0.5527787366240939
